In [1]:
# Source File: https://www.kaggle.com/datasets/shuyangli94/foodcom-recipes-with-search-terms-and-tags?resource=download
# Instead of 400K recipes, only 50K for the purpose of demonstrating our RAG model

In [2]:
!pip install pandas numpy faiss-cpu sentence-transformers torch huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [37]:
import html
import unicodedata
import re
import pandas as pd
import numpy as np
import urllib.parse
import faiss
import json
import sqlite3
from typing import List, Dict, Tuple, Any, Set, Optional
from sentence_transformers import SentenceTransformer
import json
import torch
import textwrap
from huggingface_hub import login
from huggingface_hub import InferenceClient
from google.colab import drive

In [4]:
drive.mount('/content/drive', force_remount=True)

# Path to the FAISS vector index file (stores vectors for search)
faiss_path = "/content/drive/MyDrive/Gen AI Project/recipe_faiss_index.index"

Mounted at /content/drive


In [5]:
login()

## 1. EDA

In [6]:
file_path = "/content/drive/MyDrive/Gen AI Project/50K recipes.csv"
#file_path = r"C:\Users\dolly\Desktop\Modules\Project Implementation\50K recipes.csv"

column_dtypes = {
    "id": str,
    "name": str,
    "description": str,
    "ingredients": str,
    "ingredients_raw_str": str,
    "serving_size": str,
    "servings": "Int64",
    "steps": str,
    "tags": str,
    "search_terms": str
}

df = pd.read_csv(file_path, dtype=column_dtypes)
df.head()


,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms
0,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","['onion', 'red bell pepper', 'garlic cloves', ...","[""1 medium onion, chopped coarse "",""1 m...",1 (366 g),4,"['In a food processor, pulse the onion, red pe...","['60-minutes-or-less', 'time-to-make', 'course...","{'dinner', 'shrimp'}"
1,41090,black-and-white bean salad,NaN,"['white beans', 'canned black beans', 'tomatoe...","[""1 cup canned white beans, rinsed and dra...",1 (807 g),1,"['In a large bowl, combine beans, tomato, onio...","['15-minutes-or-less', 'time-to-make', 'course...","{'vegetarian', 'salad', 'side', 'dinner', 'veg..."
2,60656,Crock Pot Italian Zucchini,This is a good recipe for weight watchers. It ...,"['zucchini', 'yellow squash', 'diced tomatoes'...","[""2 zucchini, sliced "",""2 small yel...",1 (244 g),4,['Put all ingredients in the crock pot and coo...,"['weeknight', 'time-to-make', 'course', 'main-...","{'side', 'vegetarian', 'italian'}"
3,232047,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"['beef stew meat', 'flour', 'salt', 'allspice'...","[""3 lbs beef stew meat"",""3 tablespoons ...",1 (358 g),8,"['Preheat oven to 350°F.', ""Cut beef into 1 in...","['time-to-make', 'course', 'main-ingredient', ...",{'dinner'}
4,232050,Hot Sweet Almond Brittle,This is one of our standard holiday gift recip...,"['slivered almonds', 'cider vinegar', 'sugar',...","[""12 ounces slivered almonds"",""1/4 cup ...",1 (832 g),1,['Preheat oven to 375°F Place almonds in sing...,"['time-to-make', 'course', 'preparation', 'des...",{'dessert'}


In [7]:
data_summary = pd.DataFrame({
    "Missing Values": df.isnull().sum(),
    "Remaining Count": df.notnull().sum()
})

data_summary["% Missing"] = (data_summary["Missing Values"] / len(df)) * 100
data_summary


,Missing Values,Remaining Count,% Missing
id,0,50000,0.000
name,0,50000,0.000
description,1789,48211,3.578
ingredients,0,50000,0.000
ingredients_raw_str,0,50000,0.000
serving_size,0,50000,0.000
servings,0,50000,0.000
steps,0,50000,0.000
tags,0,50000,0.000
search_terms,0,50000,0.000


In [8]:
# Check for duplicate rows
df.duplicated().sum()


np.int64(0)

## 2. Preprocess Dataset

In [9]:
# Create copy for raw dataset
df_preprocessed = df.copy()
df_preprocessed.drop(columns=["description"], inplace=True)
df_preprocessed.head()

,id,name,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms
0,232037,Simple Shrimp and Andouille Jambalaya,"['onion', 'red bell pepper', 'garlic cloves', ...","[""1 medium onion, chopped coarse "",""1 m...",1 (366 g),4,"['In a food processor, pulse the onion, red pe...","['60-minutes-or-less', 'time-to-make', 'course...","{'dinner', 'shrimp'}"
1,41090,black-and-white bean salad,"['white beans', 'canned black beans', 'tomatoe...","[""1 cup canned white beans, rinsed and dra...",1 (807 g),1,"['In a large bowl, combine beans, tomato, onio...","['15-minutes-or-less', 'time-to-make', 'course...","{'vegetarian', 'salad', 'side', 'dinner', 'veg..."
2,60656,Crock Pot Italian Zucchini,"['zucchini', 'yellow squash', 'diced tomatoes'...","[""2 zucchini, sliced "",""2 small yel...",1 (244 g),4,['Put all ingredients in the crock pot and coo...,"['weeknight', 'time-to-make', 'course', 'main-...","{'side', 'vegetarian', 'italian'}"
3,232047,Beef Stew With Dried Cherries,"['beef stew meat', 'flour', 'salt', 'allspice'...","[""3 lbs beef stew meat"",""3 tablespoons ...",1 (358 g),8,"['Preheat oven to 350°F.', ""Cut beef into 1 in...","['time-to-make', 'course', 'main-ingredient', ...",{'dinner'}
4,232050,Hot Sweet Almond Brittle,"['slivered almonds', 'cider vinegar', 'sugar',...","[""12 ounces slivered almonds"",""1/4 cup ...",1 (832 g),1,['Preheat oven to 375°F Place almonds in sing...,"['time-to-make', 'course', 'preparation', 'des...",{'dessert'}


In [10]:
# Clean ingredients column: remove single quotes and square brackets

def parse_column(value, newline=False):
    if pd.isnull(value):
        return ""

    # If it's a list or set, join items
    if isinstance(value, (list, set)):
        items = [str(i).strip() for i in value]
    elif isinstance(value, str) and value.startswith("[") and value.endswith("]"):
        try:
            items = value.strip("[]").split("','")
            items = [item.strip(" '\"") for item in items]
        except:
            items = [value]
    else:
        items = [str(value)]

    return "\n".join(items) if newline else ", ".join(items)

# Clean ingredients_raw_str: newline format + replace double quotes with single quotes
df_preprocessed["ingredients_raw_str"] = (
    df_preprocessed["ingredients_raw_str"]
    .apply(lambda x: parse_column(x, newline=True))
    .str.replace('"', "'", regex=False)  # Replace double quotes with single
    .str.replace(r"',\s*'", ", ", regex=True)  # Replace ',' pattern between quotes
    .str.replace(r"'(?!s\b)", "", regex=True)  # removes all ' except those in 's

    .str.strip()
)

# Clean tags
df_preprocessed["ingredients"] = (
    df_preprocessed["ingredients"]
    .apply(lambda x: parse_column(x))
    .str.replace('"', "", regex=False)
    .str.replace("[", "", regex=False)
    .str.replace("]", "", regex=False)
    .str.replace(r"',\s*'", ", ", regex=True)  # Optional, in case it's present
    .str.strip()
)

# Clean steps
df_preprocessed["steps"] = (
    df_preprocessed["steps"]
    .apply(lambda x: parse_column(x, newline=True))
    .str.replace('"', "'", regex=False)
    .str.replace(r"',\s*'", ", ", regex=True)  # Replace ',' pattern between quotes
    .str.strip()
)

# Clean tags
df_preprocessed["tags"] = (
    df_preprocessed["tags"]
    .apply(lambda x: parse_column(x))
    .str.replace('"', "", regex=False)
    .str.replace("[", "", regex=False)
    .str.replace("]", "", regex=False)
    .str.replace(r"',\s*'", ", ", regex=True)  # Optional, in case it's present
    .str.strip()
)

df_preprocessed["search_terms"] = (
    df_preprocessed["search_terms"]
    .apply(lambda x: parse_column(x))  # Flatten if stored as list/dict-like string
    .str.replace("{", "", regex=False)
    .str.replace("}", "", regex=False)
    .str.replace("'", " ", regex=False)
    .str.replace(r"\s*,\s*", ",", regex=True)  # Optional: ensure consistent commas
    .str.strip()
)

# Normalize comma spacing across relevant columns
columns_to_fix_commas = ["ingredients", "ingredients_raw_str", "steps", "tags", "search_terms"]

for col in columns_to_fix_commas:
    df_preprocessed[col] = (
        df_preprocessed[col]
        .str.replace(r"\s*,\s*", ", ", regex=True)  # Ensure 1 space after each comma
        .str.replace(r"\s{2,}", " ", regex=True)     # Collapse double spaces
        .str.strip()
    )

# Lowercase all string (object) columns
for col in df_preprocessed.select_dtypes(include="object").columns:
    df_preprocessed[col] = df_preprocessed[col].str.lower().str.strip()

# Replace °f with degrees fahrenheit in all text columns
for col in df_preprocessed.select_dtypes(include="object").columns:
    df_preprocessed[col] = df_preprocessed[col].str.replace("°f", " degrees fahrenheit", regex=False)



df_preprocessed.head(2)

,id,name,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms
0,232037,simple shrimp and andouille jambalaya,"onion, red bell pepper, garlic cloves, large s...","1 medium onion, chopped coarse, 1 medium red b...",1 (366 g),4,"in a food processor, pulse the onion, red pepp...","60-minutes-or-less, time-to-make, course, main...","dinner, shrimp"
1,41090,black-and-white bean salad,"white beans, canned black beans, tomatoes, oni...","1 cup canned white beans, rinsed and drained, ...",1 (807 g),1,"in a large bowl, combine beans, tomato, onion ...","15-minutes-or-less, time-to-make, course, main...","vegetarian, salad, side, dinner, vegan"


In [11]:
# Decode HTML entities and URL-encoded characters in all string columns

def decode_html_and_url(text):
    if pd.isnull(text):
        return ""
    text = urllib.parse.unquote(str(text))
    text = html.unescape(html.unescape(text))
    return text

for col in df_preprocessed.select_dtypes(include="object").columns:
    df_preprocessed[col] = df_preprocessed[col].apply(decode_html_and_url)


df_preprocessed.head(1).T

,0
id,232037
name,simple shrimp and andouille jambalaya
ingredients,"onion, red bell pepper, garlic cloves, large s..."
ingredients_raw_str,"1 medium onion, chopped coarse, 1 medium red b..."
serving_size,1 (366 g)
servings,4
steps,"in a food processor, pulse the onion, red pepp..."
tags,"60-minutes-or-less, time-to-make, course, main..."
search_terms,"dinner, shrimp"


In [12]:
# Check for extra whitespace
def has_extra_whitespace(text):
    return bool(re.search(r"\s{2,}|\t|\n", str(text)))  # 2+ spaces, tabs, or newlines

# Check across all object columns
for col in df_preprocessed.select_dtypes(include="object").columns:
    count = df_preprocessed[col].apply(has_extra_whitespace).sum()
    if count > 0:
        print(f"Column '{col}' has {count} rows with extra whitespace.")


Column 'name' has 533 rows with extra whitespace.
Column 'steps' has 22 rows with extra whitespace.


In [13]:
# Final cleanup: remove extra whitespace across all object columns
for col in df_preprocessed.select_dtypes(include="object").columns:
    df_preprocessed[col] = (
        df_preprocessed[col]
        .str.replace(r"\s{2,}", " ", regex=True)  # Collapse 2+ spaces into 1
        .str.replace(r"\t|\n", " ", regex=True)   # Remove tabs and newlines
        .str.strip()                              # Trim leading/trailing spaces
    )

# Optional: re-check to confirm whitespace is gone
for col in df_preprocessed.select_dtypes(include="object").columns:
    count = df_preprocessed[col].apply(has_extra_whitespace).sum()
    if count > 0:
        print(f"{col}: still has {count} rows with extra whitespace.")
    else:
        print(f"{col}: do not have extra spacing.")


id: do not have extra spacing.
name: do not have extra spacing.
ingredients: do not have extra spacing.
ingredients_raw_str: do not have extra spacing.
serving_size: do not have extra spacing.
steps: do not have extra spacing.
tags: do not have extra spacing.
search_terms: do not have extra spacing.


In [14]:
#Determine the tags

# First, explode the tags column into separate rows
tags_series = df_preprocessed['search_terms'].apply(lambda x: x.split(', ')).explode()

# Remove duplicates and get the count of each tag
tags_counts = tags_series.value_counts()

# Display the top 10 most frequent tags
top_10_tags = tags_counts.head(10)
top_10_tags

,count
search_terms,
dinner,22527
dessert,11922
low-sodium,9727
low-carb,9338
low-calorie,7962
healthy,7273
vegetarian,7100
side,5360
lunch,5280


## 3. Select Columns to Embed for Retrieval

In [15]:
# Combine relevant fields into a single text field for embedding

df_preprocessed["rag_text"] = (
    df_preprocessed["name"] + ". " +
    df_preprocessed["ingredients"] + ". " +
    df_preprocessed["ingredients_raw_str"] + ". " +
    "serving size: " + df_preprocessed["serving_size"] + ", servings: " + df_preprocessed["servings"].astype(str) + ". " +
    df_preprocessed["steps"] + ". " +
    df_preprocessed["tags"] + ". " +
    df_preprocessed["search_terms"]
)
pd.set_option("display.max_colwidth", None)
df_preprocessed[["id", "name", "rag_text"]].head(2)


,id,name,rag_text
0,232037,simple shrimp and andouille jambalaya,"simple shrimp and andouille jambalaya. onion, red bell pepper, garlic cloves, large shrimp, salt, hot pepper sauce, vegetable oil, andouille sausage, long grain rice, bay leaves, diced tomatoes, clam juice, fresh parsley. 1 medium onion, chopped coarse, 1 medium red bell pepper, chopped coarse, 5 medium garlic cloves, chopped coarse, 1 lb extra large shrimp, shelled and deveined, salt, hot pepper sauce, 1 tablespoon vegetable oil, 3/4 lb andouille sausage, halved lengthwise and then cut into 1/4 inch slices, 1 1/2 cups long grain rice, 4 bay leaves, 1 (14 ounce) can diced tomatoes, briefly drained, 2 (8 ounce) bottles clam juice, 1/4 cup fresh parsley, chopped. serving size: 1 (366 g), servings: 4. in a food processor, pulse the onion, red pepper and garlic until chopped very fine, ten to twelve 1-second pulses, scraping down the sides of the bowl as necessary; set aside., season the shrimp with salt and hot pepper sauce to taste., heat the oil in a large dutch oven over medium high heat until shimmering. add the shrimp in a single layer and cook, without stirring, for 30 seconds. using tongs, flip the shrimp and cook for another 30 seconds. transfer the shrimp to a medium bowl and set aside., add the sausage to the pan and cook, stirring occasionally, until lightly browned, about 3 minutes. using a slotted spoon, transfer the sausage to a second bowl., scrape the veggies from the food processor into the empty pot and cook, stirring frequently, until softened, about 3 minutes. stir in the rice and continue to stir until the grains are coated with the fat, about 1 minute., add the bay leaves, tomatoes, clam juice, and 1 cup water and bring to a boil., stir in the sausage, cover, and reduce the heat to low., cook until the rice is tender, 17 to 20 minutes., off the heat, stir in the shrimp and parsley with a fork (do not mush the rice), cover the pot, and allow to sit for 2 minutes or until the shrimp is heated through., discard the bay leaves and adjust the seasoning with salt and hot pepper sauce to taste. serve immediately.. 60-minutes-or-less, time-to-make, course, main-ingredient, preparation, main-dish, pork, seafood, easy, shrimp, one-dish-meal, meat, pork-sausage, shellfish. dinner, shrimp"
1,41090,black-and-white bean salad,"black-and-white bean salad. white beans, canned black beans, tomatoes, onion, celery, white wine vinegar, italian parsley, table salt, black pepper, olive oil. 1 cup canned white beans, rinsed and drained, 1 cup canned black beans, rinsed and drained, 1 large tomatoes, diced, 1 small onion, diced, 1 stalk celery, diced, 1 tablespoon white wine vinegar, 2 tablespoons italian parsley, minced, 1/8 teaspoon table salt, to taste, 1/8 teaspoon black pepper, to taste, 1 -2 teaspoon olive oil, to taste. serving size: 1 (807 g), servings: 1. in a large bowl, combine beans, tomato, onion and celery., gently stir in vinegar and sprinkle with parsley; season to taste and serve.. 15-minutes-or-less, time-to-make, course, main-ingredient, cuisine, preparation, occasion, north-american, salads, side-dishes, beans, american, easy, no-cook, potluck, dinner-party, picnic, spring, summer, vegan, vegetarian, dietary, seasonal, inexpensive, black-beans, to-go, number-of-servings, 3-steps-or-less, technique. vegetarian, salad, side, dinner, vegan"


## 4. Vectorization

In [16]:
'''This has been completed and stored as "/content/drive/MyDrive/Gen AI Project/recipe_faiss.index"

from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pandas as pd
import torch
from tqdm.notebook import tqdm

# Check if we have GPU and what type
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU type: {torch.cuda.get_device_name(0)}")

# Load model and move to GPU if available
model = SentenceTransformer("all-MiniLM-L6-v2")
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))

# For A100, we can use larger batch sizes
batch_size = 512  # Much larger batch size for A100

# Generate embeddings with progress tracking
print(f"Generating embeddings for {len(df_preprocessed)} recipes...")
embeddings = []

for i in tqdm(range(0, len(df_preprocessed), batch_size)):
    # Get batch
    batch = df_preprocessed.iloc[i:i+batch_size]

    # Generate embeddings for batch (use GPU acceleration)
    batch_embeddings = model.encode(batch["rag_text"].tolist(),
                                    show_progress_bar=False,
                                    batch_size=32,  # Sub-batch for GPU memory
                                    convert_to_numpy=True)

    # Append to list
    embeddings.extend(batch_embeddings)

# Add embeddings to dataframe
df_preprocessed["embedding"] = embeddings

# Convert list of vectors to np array
embedding_matrix = np.vstack(df_preprocessed["embedding"].values)

# Create FAISS index - for A100, we can use GPU-accelerated index
dimension = embedding_matrix.shape[1]

# Create and train index
index = faiss.IndexFlatL2(dimension)  # Using L2 distance

# If you want even faster search (with slight accuracy loss):
# nlist = 100  # number of clusters
# quantizer = faiss.IndexFlatL2(dimension)
# index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)
# index.train(embedding_matrix)

# Add vectors to index
index.add(embedding_matrix)

# Save to Drive
faiss.write_index(index, "/content/drive/MyDrive/Gen AI Project/recipe_faiss.index")
df_preprocessed.to_csv("/content/drive/MyDrive/Gen AI Project/recipe_metadata.csv", index=False)

print(f"Successfully created and saved index with {len(df_preprocessed)} recipes and dimension {dimension}")

'''

'This has been completed and stored as "/content/drive/MyDrive/Gen AI Project/recipe_faiss.index"\n\nfrom sentence_transformers import SentenceTransformer\nimport numpy as np\nimport faiss\nimport pandas as pd\nimport torch\nfrom tqdm.notebook import tqdm\n\n# Check if we have GPU and what type\nprint(f"GPU available: {torch.cuda.is_available()}")\nif torch.cuda.is_available():\n    print(f"GPU type: {torch.cuda.get_device_name(0)}")\n\n# Load model and move to GPU if available\nmodel = SentenceTransformer("all-MiniLM-L6-v2")\nif torch.cuda.is_available():\n    model = model.to(torch.device("cuda"))\n\n# For A100, we can use larger batch sizes\nbatch_size = 512  # Much larger batch size for A100\n\n# Generate embeddings with progress tracking\nprint(f"Generating embeddings for {len(df_preprocessed)} recipes...")\nembeddings = []\n\nfor i in tqdm(range(0, len(df_preprocessed), batch_size)):\n    # Get batch\n    batch = df_preprocessed.iloc[i:i+batch_size]\n\n    # Generate embeddings f

## 5. Retrieval & Generation

### Retrieval Section
Your retrieval functionality implements a multi-layered search approach that effectively combines:

Semantic Search - Uses FAISS vector database to find recipes semantically similar to the user's query, leveraging embeddings created with SentenceTransformer.
Tag-Based Filtering - Pre-filters results based on dietary restrictions (gluten-free, low cholesterol, etc.) using the recipe tags from your dataset.
Ingredient Availability Matching - Prioritizes recipes that use ingredients the user already has (from SQL database) using Jaccard similarity.
Missing Ingredient Detection - Identifies which ingredients the user would need to purchase for each recipe recommendation.
Fallback Mechanism - When no good matches are found, the system can generate custom recipe suggestions based on available ingredients.

### Generation Section
Your generation functionality creates contextually appropriate responses by:

Structured Context Building - Formats retrieved recipes and their details into a structured prompt for the LLM.
Personalized Response Creation - Uses Llama 3.2 to generate natural, conversational responses that address the user's specific needs.
Dietary Awareness - Explicitly acknowledges dietary restrictions and explains how recommendations meet those requirements.
Missing Ingredient Communication - Clearly indicates which additional ingredients would be needed.
Helpful Additions - Includes relevant cooking tips and offers to provide more detailed information.

In [17]:
# Path configurations
faiss_path = "/content/drive/MyDrive/Gen AI Project/recipe_faiss.index"
recipe_metadata_path = "/content/drive/MyDrive/Gen AI Project/recipe_metadata.csv"

In [25]:
def parse_instructions(recipe):
    steps = recipe.get('steps', "")
    if isinstance(steps, list):
        return steps
    if isinstance(steps, str):
        import re, ast
        if steps.startswith("[") and steps.endswith("]"):
            try:
                return ast.literal_eval(steps)
            except:
                pass
        if '\n' in steps:
            return [step.strip() for step in steps.split('\n') if step.strip()]
        numbered_steps = re.findall(r'\d+\.\s*([^\d]+?)(?=\d+\.|$)', steps)
        if numbered_steps and len(numbered_steps) > 1:
            return [step.strip() for step in numbered_steps]
        return [s.strip() for s in re.split(r'(?<!\d)\.(?!\d)', steps) if s.strip()]
    return []

def get_recipe_recommendations(fridge_items: Dict[str, str], preferences: Dict[str, str],
                                faiss_path: str, recipe_metadata_path: str) -> Dict:
    available_ingredients = [item.split(':')[0].strip() if ':' in item else item.strip()
                             for item, _ in fridge_items.items()]
    recipe_metadata = pd.read_csv(recipe_metadata_path)
    index = faiss.read_index(faiss_path)

    ingredient_text = ", ".join(fridge_items.keys())
    rag_text = (
        f"Recipe with {ingredient_text}. "
        f"Preferences: mealtype: {preferences.get('mealtype', '')}, "
        f"dietaryneeds: {preferences.get('dietaryneeds', '')}, "
        f"cuisinetype: {preferences.get('cuisinetype', '')}."
    )

    model = SentenceTransformer("all-MiniLM-L6-v2")
    if torch.cuda.is_available():
        model = model.to(torch.device("cuda"))
    search_embedding = model.encode([rag_text])[0].astype(np.float32)
    distances, indices = index.search(np.array([search_embedding]), 5)
    matching_recipes = recipe_metadata.iloc[indices[0]].to_dict('records')

    best_match = None
    best_score = -1
    missing_ingredients = []
    recipe_ingredients = []

    for i, recipe in enumerate(matching_recipes):
        curr_recipe_ingredients = []
        if 'ingredients' in recipe and recipe['ingredients']:
            if isinstance(recipe['ingredients'], str):
                curr_recipe_ingredients = [ing.strip() for ing in recipe['ingredients'].split(',')]
        elif 'ingredients_raw_str' in recipe and recipe['ingredients_raw_str']:
            curr_recipe_ingredients = [ing.strip() for ing in recipe['ingredients_raw_str'].split(',')]

        common = [i for i in curr_recipe_ingredients if i in available_ingredients]
        missing = [i for i in curr_recipe_ingredients if i not in available_ingredients]
        ingredient_score = len(common) / len(curr_recipe_ingredients) if curr_recipe_ingredients else 0

        preference_score = 0
        tags = recipe.get('tags', '').lower()
        if preferences.get('mealtype', '').lower() in tags:
            preference_score += 1
        if isinstance(preferences.get('dietaryneeds'), list):
            for need in preferences['dietaryneeds']:
                if need.lower() in tags:
                    preference_score += 1
        elif isinstance(preferences.get('dietaryneeds'), str):
            if preferences['dietaryneeds'].lower() in tags:
                preference_score += 1
        if preferences.get('cuisinetype', '').lower() in tags:
            preference_score += 1

        num_prefs = 1 + (len(preferences['dietaryneeds']) if isinstance(preferences['dietaryneeds'], list) else 1) + 1
        preference_score = preference_score / num_prefs
        faiss_score = 1 - (distances[0][i] / max(distances[0]) if max(distances[0]) > 0 else 0)

        overall_score = (ingredient_score * 0.5) + (preference_score * 0.3) + (faiss_score * 0.2)

        if overall_score > best_score:
            best_score = overall_score
            best_match = recipe
            missing_ingredients = missing
            recipe_ingredients = curr_recipe_ingredients

    instructions = parse_instructions(best_match)
    instructions = [step.lstrip(" ,") for step in instructions]

    return {
        "recipe_name": best_match.get("name", "Unknown Recipe"),
        "customized_for": preferences,
        "ingredients_needed": recipe_ingredients,
        "instructions": instructions,
        "missing_ingredients": missing_ingredients,
        "match_score": best_score,
        "matching_recipes": matching_recipes
    }

if __name__ == "__main__":
    # Example fridge items
    fridge_items = {
        "oranges": "3",
        "lemon": "4",
        "sugar": "",
        "olive oil": "",
        "salt": "",
        "low-fat milk": "",
        "cocoa powder": "",
        "banana": "4",
        "eggs": "10"
    }

    # Example preferences
    preferences = {
        'mealtype': 'breakfast',
    'dietaryneeds': ['diabetic', 'low-calorie', 'low-sodium'],
    'cuisinetype': 'french'
}

    # Paths to data files
    faiss_path = "/content/drive/MyDrive/Gen AI Project/recipe_faiss.index"
    recipe_metadata_path = "/content/drive/MyDrive/Gen AI Project/recipe_metadata.csv"

    # Get recipe recommendation
    recommendation = get_recipe_recommendations(
        fridge_items,
        preferences,
        faiss_path,
        recipe_metadata_path
    )

    # Display results
    print(f"Recipe Name: {recommendation['recipe_name']}")
    print(f"Customised for: {recommendation['customized_for']}")
    print("Ingredients needed:")
    for ingredient in recommendation['ingredients_needed']:
        print(f"- {ingredient}")
    print("\nInstructions:")
    for i, step in enumerate(recommendation['instructions'], 1):
        print(f"{i}. {step}")

    if recommendation['missing_ingredients']:
        print("\nMissing ingredients:")
        for ingredient in recommendation['missing_ingredients']:
            print(f"- {ingredient}")

Recipe Name: orange omelette for harlots and ruffians
Customised for: {'mealtype': 'breakfast', 'dietaryneeds': ['diabetic', 'low-calorie', 'low-sodium'], 'cuisinetype': 'french'}
Ingredients needed:
- eggs
- oranges
- lemon
- sugar
- olive oil
- salt

Instructions:
1. juice the oranges and the lemon
2. beat the eggs, add the juice, the sugar, and salt to taste, and cook the omelette in olive oil
3. serve warm


In [26]:
#3
def build_prompt(context: str, fridge_items: Dict, preferences: Dict) -> str:
    # Format ingredients with quantities on separate lines for clarity
    ingredients_list = []
    for item, qty in fridge_items.items():
        if qty and str(qty).strip():
            ingredients_list.append(f"- {item}: {qty}")
        else:
            ingredients_list.append(f"- {item}")

    formatted_ingredients = "\n".join(ingredients_list)

    return (
        "You are a certified dietician and culinary expert.\n"
        "Your task is to create one personalized recipe using ONLY the ingredients available in the user's fridge.\n\n"

        "User Preferences:\n"
        f"- Meal Type: {preferences['mealtype']}\n"
        f"- Dietary Needs: {', '.join(preferences['dietaryneeds']) if isinstance(preferences['dietaryneeds'], list) else preferences['dietaryneeds']}\n"
        f"- Cuisine Type: {preferences['cuisinetype']}\n\n"

        "AVAILABLE INGREDIENTS IN USER'S FRIDGE (all of these are available):\n"
        f"{formatted_ingredients}\n\n"

        "IMPORTANT INSTRUCTIONS:\n"
        "- You MUST use ONLY ingredients from the list above.\n"
        "- DO NOT mark any ingredient from the above list as 'Missing' - they are ALL available.\n"
        "- Do not invent or add any ingredients not listed above.\n"
        "- Only ingredients listed above may be used.\n"
        "- If a required ingredient is not listed (even honey, spices, etc.), you must clearly label it as 'Missing'.\n"
        "- Exception: water is allowed without marking it missing.\n"

        "- Match the preferences strictly (meal type, dietary needs, cuisine).\n"
        "- Provide clear, realistic ingredient quantities.\n\n"

        "Reference Recipe (for inspiration):\n"
        f"{context}\n\n"

        "Please generate exactly one recipe that includes:\n"
        "1. Recipe Name\n"
        "2. List of Ingredients with quantities\n"
        "3. Step-by-step Instructions\n"
        "4. Brief nutritional note explaining how this recipe meets the dietary needs"
    )

In [39]:
# Combine top-matching recipes into context

matching_recipes = recommendation["matching_recipes"]

recipe_context = "\n\n".join([
    f"{r['name']}: {r['ingredients']}. {r.get('steps', '')}" for r in matching_recipes
])

client = InferenceClient("mistralai/Mistral-7B-Instruct-v0.1")

# Generate recipe from LLM
prompt = build_prompt(recipe_context, fridge_items, preferences)
response = client.text_generation(prompt, max_new_tokens=1000, temperature=0.1)

# Cleanly display the result
if "Nutritional Note:" in response:
    # Split and format nicely
    body, note = response.split("Nutritional Note:", 1)

    # Remove accidental headers like "of the user."
    body_cleaned = body.strip().replace("of the user.", "").strip()

    print("\nRecommended Recipe:\n")
    print(body_cleaned)
    print("\nNutritional Note:")
    print(textwrap.fill(note.strip(), width=90))

else:
    print("\nRecommended Recipe:\n")
    print(response.strip())



Recommended Recipe:

Recipe Name: Low-Calorie, Low-Sodium French Breakfast

List of Ingredients with quantities:
- 2 oranges, peeled and sliced
- 1 lemon, juiced
- 1 tbsp sugar
- 2 tbsp olive oil
- 1/4 tsp salt
- 1 cup low-fat milk
- 1/4 cup cocoa powder
- 2 bananas, sliced
- 4 eggs

Step-by-step Instructions:

1. In a small bowl, whisk together the sugar, cocoa powder, and salt.
2. In a separate bowl, whisk together the eggs, milk, lemon juice, and olive oil.
3. Heat a non-stick skillet over medium heat. Add the egg mixture and cook for 2-3 minutes, or until the bottom is golden brown and the top is still runny.
4. Add the sliced oranges and bananas to the skillet.
5. Sprinkle the sugar mixture over the fruit.
6. Cook for another 1-2 minutes, or until the fruit is heated through and the sugar has dissolved.
7. Serve the French breakfast hot.

Nutritional Note:
This recipe is low-calorie, low-sodium, and diabetic-friendly. The oranges and bananas
provide natural sweetness without addi

In [43]:
#Retrieve the ID of the Top-Matched Recipe listed above for checking

top_recipe = recommendation["matching_recipes"][0]
recipe_id = top_recipe.get("id", "ID not found")
print("Matched Recipe ID:", recipe_id)

Matched Recipe ID: 20362


### **Ignore all codes below**

In [56]:
import re
from difflib import get_close_matches

def normalize_ingredient_name(text: str) -> str:
    """
    Lowercase and remove descriptors (e.g., 'chopped', 'sliced') to focus on core ingredient.
    """
    text = text.lower()
    text = re.sub(r'[^a-z\s\-]', '', text)  # remove numbers and symbols
    words = text.split()
    skip_words = {'sliced', 'peeled', 'chopped', 'minced', 'fresh', 'juiced', 'optional'}
    filtered = [w for w in words if w not in skip_words]
    return ' '.join(filtered).strip()

def extract_quantity_and_name(ingredient_line: str) -> Tuple[Optional[float], str]:
    """
    Attempt to split the quantity and the ingredient name.
    e.g., '2 oranges, peeled and sliced' => (2, 'oranges')
    """
    match = re.match(r'^([\d/.]+)\s+([a-zA-Z\s\-]+)', ingredient_line)
    if match:
        qty_str, name_part = match.groups()
        try:
            qty = eval(qty_str)  # supports fractions like "1/2"
        except:
            qty = None
        return qty, normalize_ingredient_name(name_part)
    else:
        return None, normalize_ingredient_name(ingredient_line)

def check_ingredients_used(llm_output: str, fridge_items: Dict[str, str]) -> List[str]:
    """
    Cross-check ingredients from LLM output against fridge_items.
    If quantity is specified in LLM output, it will compare actual quantity.
    Otherwise, it just checks for presence.
    """
    # Normalize fridge items
    fridge_normalized = {
        normalize_ingredient_name(k): float(v) if str(v).replace('.', '', 1).isdigit() else None
        for k, v in fridge_items.items()
    }

    # Extract ingredient block
    match = re.search(r'(?i)list of ingredients with quantities:\s*(.+?)\n\s*\n', llm_output, re.DOTALL)
    if not match:
        return []

    raw_ingredients_block = match.group(1)
    lines = [l.strip("- ").strip() for l in raw_ingredients_block.splitlines() if l.strip()]

    missing = []
    for line in lines:
        qty, name = extract_quantity_and_name(line)

        if name not in fridge_normalized:
            close = get_close_matches(name, fridge_normalized.keys(), cutoff=0.85)
            if not close:
                missing.append(line)
                continue
            name = close[0]

        fridge_qty = fridge_normalized[name]
        if qty and fridge_qty is not None and fridge_qty < qty:
            missing.append(line)  # Not enough quantity

    return missing


In [57]:
# Generate recipe from LLM
prompt = build_prompt(recipe_context, fridge_items, preferences)
response = client.text_generation(prompt, max_new_tokens=1000, temperature=0.1)

# 🔮 Print the LLM-generated recipe
if "Nutritional Note:" in response:
    body, note = response.split("Nutritional Note:", 1)
    body_cleaned = body.strip().replace("of the user.", "").strip()

    print("\n🔮 Recommended Recipe:\n")
    print(body_cleaned)
    print("\n💡 Nutritional Note:")
    print(textwrap.fill(note.strip(), width=90))

else:
    print("\n🔮 Recommended Recipe:\n")
    print(response.strip())

# ✅ THEN insert this block immediately after:
missing = check_ingredients_used(response, fridge_items)

if missing:
    print("\n🚨 Additional ingredients required to make this recipe:")
    for m in missing:
        print(f"- {m}")
else:
    print("\n✅ All ingredients used are available in the fridge.")



🔮 Recommended Recipe:

Recipe Name: Low-Calorie, Low-Sodium French Breakfast

List of Ingredients with quantities:
- 2 oranges, peeled and sliced
- 1 lemon, juiced
- 1 tbsp sugar
- 2 tbsp olive oil
- 1/4 tsp salt
- 1 cup low-fat milk
- 1/4 cup cocoa powder
- 2 bananas, sliced
- 4 eggs

Step-by-step Instructions:

1. In a small bowl, whisk together the sugar, cocoa powder, and salt.
2. In a separate bowl, whisk together the eggs, milk, lemon juice, and olive oil.
3. Heat a non-stick skillet over medium heat. Add the egg mixture and cook for 2-3 minutes, or until the bottom is golden brown and the top is still runny.
4. Add the sliced oranges and bananas to the skillet.
5. Sprinkle the sugar mixture over the fruit.
6. Cook for another 1-2 minutes, or until the fruit is heated through and the sugar has dissolved.
7. Serve the French breakfast hot.

💡 Nutritional Note:
This recipe is low-calorie, low-sodium, and diabetic-friendly. The oranges and bananas
provide natural sweetness without 

## Prompt Engineeering - Trial and Error

In [ ]:
def build_prompt(context: str, fridge_items: Dict, preferences: Dict) -> str:
    return (
        "You are a certified dietician and culinary expert.\n"
        "Your task is to create one personalized recipe based on the user's available ingredients and dietary preferences.\n\n"

        "User Preferences:\n"
        f"- Meal Type: {preferences['mealtype']}\n"
        f"- Dietary Needs: {', '.join(preferences['dietaryneeds']) if isinstance(preferences['dietaryneeds'], list) else preferences['dietaryneeds']}\n"
        f"- Cuisine Type: {preferences['cuisinetype']}\n\n"

        "User's available ingredients:\n"
        f"{', '.join(fridge_items.keys())}\n\n"

        "Constraints:\n"
        "- ONLY use ingredients listed in user's available ingredients.\n"
        "- If any extra ingredients are needed, clearly mark them as 'Missing'.\n"
        "- Match the preferences strictly (meal type, dietary needs, cuisine).\n"
        "- Provide clear, realistic ingredient quantities.\n"
        "- Keep instructions simple and step-by-step.\n\n"

        "Reference Recipe (for inspiration):\n"
        f"{context}\n\n"

        "Please generate exactly one recipe that includes:\n"
        "1. Recipe Name\n"
        "2. List of Ingredients (include quantities and label any 'Missing')\n"
        "3. Step-by-step Instructions\n"
        "4. At the end, state if any ingredients are 'Missing' from the fridge."
    )


In [ ]:
# Combine top-matching recipes into context
recipe_context = "\n\n".join([
    f"{r['name']}: {r['ingredients']}. {r.get('steps', '')}" for r in matching_recipes
])

# Generate the recipe using Mistral via HF Hub
prompt = build_prompt(recipe_context, fridge_items, preferences)
response = client.text_generation(prompt, max_new_tokens=1000, temperature=0.1)

# Print the LLM-generated recipe
print("\nRecommended Recipe:")
print(response)


In [ ]:
#1
def build_prompt(context: str, fridge_items: Dict, preferences: Dict) -> str:
    # Format ingredients with quantities on separate lines for clarity
    ingredients_list = []
    for item, qty in fridge_items.items():
        if qty and str(qty).strip():
            ingredients_list.append(f"- {item}: {qty}")
        else:
            ingredients_list.append(f"- {item}")

    formatted_ingredients = "\n".join(ingredients_list)

    return (
        "You are a certified dietician and culinary expert.\n"
        "Your task is to create one personalized recipe using ONLY the ingredients available in the user's fridge.\n\n"

        "User Preferences:\n"
        f"- Meal Type: {preferences['mealtype']}\n"
        f"- Dietary Needs: {', '.join(preferences['dietaryneeds']) if isinstance(preferences['dietaryneeds'], list) else preferences['dietaryneeds']}\n"
        f"- Cuisine Type: {preferences['cuisinetype']}\n\n"

        "AVAILABLE INGREDIENTS IN USER'S FRIDGE (all of these are available):\n"
        f"{formatted_ingredients}\n\n"

        "IMPORTANT INSTRUCTIONS:\n"
        "- You MUST use ONLY ingredients from the list above.\n"
        "- DO NOT mark any ingredient from the above list as 'Missing' - they are ALL available.\n"
        "- Do not invent or add any ingredients not listed above.\n"
        "- If you absolutely need a basic ingredient not listed (like water), you can include it without marking it missing.\n"
        "- Match the preferences strictly (meal type, dietary needs, cuisine).\n"
        "- Provide clear, realistic ingredient quantities.\n\n"

        "Reference Recipe (for inspiration):\n"
        f"{context}\n\n"

        "Please generate exactly one recipe that includes:\n"
        "1. Recipe Name\n"
        "2. List of Ingredients with quantities\n"
        "3. Step-by-step Instructions\n"
        "4. Brief nutritional note explaining how this recipe meets the dietary needs"
    )

In [ ]:
# Combine top-matching recipes into context
recipe_context = "\n\n".join([
    f"{r['name']}: {r['ingredients']}. {r.get('steps', '')}" for r in matching_recipes
])

# Generate the recipe using Mistral via HF Hub
prompt = build_prompt(recipe_context, fridge_items, preferences)
response = client.text_generation(prompt, max_new_tokens=1000, temperature=0.1)

# Print the LLM-generated recipe
print("\nRecommended Recipe:")
print(response)

In [ ]:
#2
def build_prompt(context: str, fridge_items: Dict, preferences: Dict) -> str:
    # Format ingredients with quantities on separate lines for clarity
    ingredients_list = []
    for item, qty in fridge_items.items():
        if qty and str(qty).strip():
            ingredients_list.append(f"- {item}: {qty}")
        else:
            ingredients_list.append(f"- {item}")

    formatted_ingredients = "\n".join(ingredients_list)

    return (
        "You are a certified dietician and culinary expert.\n"
        "Your task is to create one personalized recipe using ONLY the ingredients available in the user's fridge.\n\n"

        "User Preferences:\n"
        f"- Meal Type: {preferences['mealtype']}\n"
        f"- Dietary Needs: {', '.join(preferences['dietaryneeds']) if isinstance(preferences['dietaryneeds'], list) else preferences['dietaryneeds']}\n"
        f"- Cuisine Type: {preferences['cuisinetype']}\n\n"

        "AVAILABLE INGREDIENTS IN USER'S FRIDGE (all of these are available):\n"
        f"{formatted_ingredients}\n\n"

        "IMPORTANT INSTRUCTIONS:\n"
        "- You MUST use ONLY ingredients from the list above.\n"
        "- DO NOT mark any ingredient from the above list as 'Missing' - they are ALL available.\n"
        "- Do not invent or add any ingredients not listed above.\n"
        "- Only ingredients listed above may be used.\n"
        "- If a required ingredient is not listed (even honey, spices, etc.), you must clearly label it as 'Missing'.\n"
        "- Exception: water is allowed without marking it missing.\n"

        "- Match the preferences strictly (meal type, dietary needs, cuisine).\n"
        "- Provide clear, realistic ingredient quantities.\n\n"

        "Reference Recipe (for inspiration):\n"
        f"{context}\n\n"

        "Please generate exactly one recipe that includes:\n"
        "1. Recipe Name\n"
        "2. List of Ingredients with quantities\n"
        "3. Step-by-step Instructions\n"
        "4. Brief nutritional note explaining how this recipe meets the dietary needs"
    )

In [ ]:
# Combine top-matching recipes into context
recipe_context = "\n\n".join([
    f"{r['name']}: {r['ingredients']}. {r.get('steps', '')}" for r in matching_recipes
])

# Generate the recipe using Mistral via HF Hub
prompt = build_prompt(recipe_context, fridge_items, preferences)
response = client.text_generation(prompt, max_new_tokens=1000, temperature=0.1)

# Print the LLM-generated recipe
print("\nRecommended Recipe:\n")
print(response)

In [ ]:
#3
def build_prompt(context: str, fridge_items: Dict, preferences: Dict) -> str:
    # Format ingredients with quantities on separate lines for clarity
    ingredients_list = []
    for item, qty in fridge_items.items():
        if qty and str(qty).strip():
            ingredients_list.append(f"- {item}: {qty}")
        else:
            ingredients_list.append(f"- {item}")

    formatted_ingredients = "\n".join(ingredients_list)

    return (
        "You are a certified dietician and culinary expert.\n"
        "Your task is to create one personalized recipe using ONLY the ingredients available in the user's fridge.\n\n"

        "User Preferences:\n"
        f"- Meal Type: {preferences['mealtype']}\n"
        f"- Dietary Needs: {', '.join(preferences['dietaryneeds']) if isinstance(preferences['dietaryneeds'], list) else preferences['dietaryneeds']}\n"
        f"- Cuisine Type: {preferences['cuisinetype']}\n\n"

        "AVAILABLE INGREDIENTS IN USER'S FRIDGE (all of these are available):\n"
        f"{formatted_ingredients}\n\n"

        "IMPORTANT INSTRUCTIONS:\n"
        "- You MUST use ONLY ingredients from the list above.\n"
        "- DO NOT mark any ingredient from the above list as 'Missing' - they are ALL available.\n"
        "- Do not invent or add any ingredients not listed above.\n"
        "- Only ingredients listed above may be used.\n"
        "- If a required ingredient is not listed (even honey, spices, etc.), you must clearly label it as 'Missing'.\n"
        "- Exception: water is allowed without marking it missing.\n"

        "- Match the preferences strictly (meal type, dietary needs, cuisine).\n"
        "- Provide clear, realistic ingredient quantities.\n\n"

        "Reference Recipe (for inspiration):\n"
        f"{context}\n\n"

        "Please generate exactly one recipe that includes:\n"
        "1. Recipe Name\n"
        "2. List of Ingredients with quantities\n"
        "3. Step-by-step Instructions\n"
        "4. Brief nutritional note explaining how this recipe meets the dietary needs"
    )

In [ ]:
import textwrap

# Generate recipe from LLM
prompt = build_prompt(recipe_context, fridge_items, preferences)
response = client.text_generation(prompt, max_new_tokens=1000, temperature=0.1)

# Cleanly display the result
if "Nutritional Note:" in response:
    # Split and format nicely
    body, note = response.split("Nutritional Note:", 1)

    # Remove accidental headers like "of the user."
    body_cleaned = body.strip().replace("of the user.", "").strip()

    print("\nRecommended Recipe:\n")
    print(body_cleaned)
    print("\nNutritional Note:")
    print(textwrap.fill(note.strip(), width=90))

else:
    print("\nRecommended Recipe:\n")
    print(response.strip())


In [23]:
def build_prompt(context: str, fridge_items: Dict, preferences: Dict) -> str:
    # Format ingredients with quantities on separate lines for clarity
    ingredients_list = []
    for item, qty in fridge_items.items():
        if qty and str(qty).strip():
            ingredients_list.append(f"- {item}: {qty}")
        else:
            ingredients_list.append(f"- {item}")

    formatted_ingredients = "\n".join(ingredients_list)

    return (
        "You are a certified dietician and culinary expert.\n"
        "Your task is to examine the reference recipes and create a personalized recipe based on available ingredients.\n\n"

        "User Preferences:\n"
        f"- Meal Type: {preferences['mealtype']}\n"
        f"- Dietary Needs: {', '.join(preferences['dietaryneeds']) if isinstance(preferences['dietaryneeds'], list) else preferences['dietaryneeds']}\n"
        f"- Cuisine Type: {preferences['cuisinetype']}\n\n"

        "Available Ingredients in User's Fridge:\n"
        f"{formatted_ingredients}\n\n"

        "Reference Recipes (in order of relevance):\n"
        f"{context}\n\n"

        "Instructions:\n"
        "1. First, check if the most relevant reference recipe can be made with ONLY the ingredients in the user's fridge.\n"
        "2. If yes, adapt that recipe and provide it.\n"
        "3. If no, adapt the next closest recipe that can be made with available ingredients.\n"
        "4. If a recipe requires ingredients not in the fridge, clearly label those ingredients as '(MISSING)' in your ingredients list.\n"
        "5. Try to minimize the number of missing ingredients.\n"
        "6. Ensure the recipe meets the user's dietary needs and preferences.\n\n"

        "Please generate one recipe that includes:\n"
        "1. Recipe Name\n"
        "2. List of Ingredients with quantities (label any not in the fridge as '(MISSING)')\n"
        "3. Step-by-step Instructions\n"
        "4. Missing Ingredients Summary (list only the ingredients that aren't available in the fridge)\n"
    )

In [32]:
# Combine top-matching recipes into context
recipe_context = "\n\n".join([
    f"{r['name']}: {r['ingredients']}. {r.get('steps', '')}" for r in matching_recipes
])

# Generate the recipe using Mistral via HF Hub
prompt = build_prompt(recipe_context, fridge_items, preferences)
response = client.text_generation(prompt, max_new_tokens=1000, temperature=0.1)

# Print the LLM-generated recipe
print("\nRecommended Recipe:")
print(response)

NameError: name 'matching_recipes' is not defined

In [ ]:
def build_prompt(context: str, fridge_items: Dict, preferences: Dict) -> str:
    # Format ingredients with quantities on separate lines for clarity
    ingredients_list = []
    for item, qty in fridge_items.items():
        if qty and str(qty).strip():
            ingredients_list.append(f"- {item}: {qty}")
        else:
            ingredients_list.append(f"- {item}")

    # Create a comma-separated list of ingredients for easy reference
    ingredients_simple = ", ".join(fridge_items.keys())

    formatted_ingredients = "\n".join(ingredients_list)

    return (
        "You are a certified dietician and culinary expert.\n"
        "Your task is to create a personalized recipe using ONLY the ingredients listed below.\n\n"

        "User Preferences:\n"
        f"- Meal Type: {preferences['mealtype']}\n"
        f"- Dietary Needs: {', '.join(preferences['dietaryneeds']) if isinstance(preferences['dietaryneeds'], list) else preferences['dietaryneeds']}\n"
        f"- Cuisine Type: {preferences['cuisinetype']}\n\n"

        "AVAILABLE INGREDIENTS (use ONLY these, nothing else):\n"
        f"{formatted_ingredients}\n\n"
        f"The ONLY ingredients you can use are: {ingredients_simple}\n\n"

        "Reference Recipes (for inspiration):\n"
        f"{context}\n\n"

        "STRICT RULES:\n"
        "1. You may ONLY use ingredients from the 'AVAILABLE INGREDIENTS' list above.\n"
        "2. If you mention ANY ingredient not in the list above, mark it as '(MISSING)'.\n"
        "3. Do NOT add any ingredients like honey, cereal, flour, etc. unless they are specifically listed above.\n"
        "4. The recipe must match the user's preferences (meal type, dietary needs, cuisine).\n"
        "5. If you're unsure if an ingredient is available, assume it is NOT and mark it as '(MISSING)'.\n\n"

        "Generate ONE recipe with:\n"
        "1. Recipe Name\n"
        "2. Ingredients List (with quantities, mark any not from the available list as '(MISSING)')\n"
        "3. Step-by-step Instructions\n"
        "4. Missing Ingredients Summary (list ONLY ingredients that aren't in the available list)\n"
    )

In [ ]:
# Combine top-matching recipes into context
recipe_context = "\n\n".join([
    f"{r['name']}: {r['ingredients']}. {r.get('steps', '')}" for r in matching_recipes
])

# Generate the recipe using Mistral via HF Hub
prompt = build_prompt(recipe_context, fridge_items, preferences)
response = client.text_generation(prompt, max_new_tokens=1000, temperature=0.3)

# Print the LLM-generated recipe
print("\nRecommended Recipe:")
print(response)

